In [ ]:
%pip install tensorflow keras nltk scikit-learn transformers


In [ ]:
pip install --upgrade tensorflow

In [ ]:
pip install textblob vaderSentiment


In [ ]:
pip install matplotlib

In [ ]:
pip uninstall transformers -y


In [ ]:
pip install google-generativeai

In [ ]:
pip install nltk

In [ ]:
pip install transformers tensorflow torch scikit-learn matplotlib

In [ ]:
pip cache purge

In [ ]:

pip install keras==2.13.1  transformers==4.33.0

In [ ]:
pip show tensorflow keras transformers

In [ ]:
pip install keras==3.0.5 transformers==4.38.1

In [ ]:
pip install tensorflow-addons


In [ ]:
#Another Version

In [ ]:
pip install sentence-transformers

In [ ]:
#Mood_keywords.json
import json
from collections import defaultdict, Counter

# ---- Step 1: Load original mood_keywords.json (with Mood_0, Mood_1...) ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

# ---- Step 2: Map Mood_# to actual mood names ----
mood_map = {
    "Mood_0": "Abuse",
    "Mood_1": "Sexual Issues",
    "Mood_2": "Erectile Dysfunction",
    "Mood_3": "Relationship Issues",
    "Mood_4": "Depression",
    "Mood_5": "Child Custody",
    "Mood_6": "Parental Issues",
    "Mood_7": "Infidelity",
    "Mood_8": "Marriage Problems",
    "Mood_9": "Motherhood",
    "Mood_10": "Gender Identity",
    "Mood_11": "Therapist Issues",
    "Mood_12": "Therapy Anxiety",
    "Mood_13": "Marital Issues",
    "Mood_14": "Unrequited Love",
    "Mood_15": "Therapy Nervousness",
    "Mood_16": "Health Trauma",
    "Mood_17": "PTSD",
    "Mood_18": "Alcoholism",
    "Mood_19": "Breakups",
    "Mood_20": "Sexless Marriage",
    "Mood_21": "Hearing Voices",
    "Mood_22": "Eating Disorders",
    "Mood_23": "Suicidal Thoughts",
    "Mood_24": "Panic Attacks",
    "Mood_25": "Toxic Relationships",
    "Mood_26": "Anxiety And Depression",
    "Mood_27": "Daughter Stress",
    "Mood_28": "Work And Family Stress",
    "Mood_29": "School Stress"
}

# Rename original mood keys
renamed_keywords = {}
for mood_id, keywords in mood_keywords.items():
    readable_label = mood_map.get(mood_id, mood_id)
    renamed_keywords[readable_label] = keywords

# ---- Step 3: Define broader categories and map moods under them ----
category_map = {
    "Depression": [
        "Depression", "Unrequited Love", "Breakups", "Suicidal Thoughts", "Anxiety And Depression"
    ],
    "Anxiety": [
        "Therapy Anxiety", "Therapy Nervousness", "Panic Attacks", "Hearing Voices", "Anxiety And Depression"
    ],
    "Stress": [
        "Work And Family Stress", "Daughter Stress", "School Stress", "Toxic Relationships",
        "Parental Issues", "Child Custody"
    ],
    "Health Trauma": [
        "Health Trauma", "Eating Disorders", "Alcoholism", "PTSD"
    ],
    "Panic Attack": [
        "Panic Attacks"
    ],
    "Work and School Stress": [
        "Work And Family Stress", "School Stress"
    ],
    "Relationship Issues": [
        "Marriage Problems", "Infidelity", "Sexless Marriage", "Marital Issues", "Relationship Issues"
    ],
    "Sexual Health": [
        "Erectile Dysfunction", "Sexual Issues"
    ],
    "Identity": [
        "Gender Identity", "Motherhood"
    ],
    "Therapy Issues": [
        "Therapist Issues", "Therapy Anxiety", "Therapy Nervousness"
    ],
    "Abuse": [
        "Abuse"
    ]
}

# ---- Step 4: Group and merge keywords for broader categories ----
grouped_keywords = {}

for broad_mood, specific_moods in category_map.items():
    combined_keywords = []
    for mood in specific_moods:
        combined_keywords.extend(renamed_keywords.get(mood, []))
    
    # Count and deduplicate
    counter = Counter(combined_keywords)
    grouped_keywords[broad_mood] = [kw for kw, _ in counter.most_common(20)]

# ---- Step 5: Save the final grouped mood keywords ----
with open("mood_keywords.json", "w", encoding="utf-8") as f:
    json.dump(grouped_keywords, f, indent=2, ensure_ascii=False)

print(f"✅ mood_keywords.json created with {len(grouped_keywords)} broader categories.")

In [ ]:
#This Code uses Mood_keywords.json file
import os
import json
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model,save_model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Layer, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]
contexts = [sample["Context"] for sample in parsed_data]

# ---- Load Mood Keywords ----
with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

def detect_mood(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

X_texts, Y_labels = [], []
for context in contexts:
    mood = detect_mood(context)
    if mood:
        X_texts.append(context)
        Y_labels.append(mood)

# ---- Encode and Filter Labels ----
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_labels)
with open("mood_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

label_counts = Counter(Y_encoded)
valid_labels = {label for label, count in label_counts.items() if count > 1}
filtered = [(x, y) for x, y in zip(X_texts, Y_encoded) if y in valid_labels]
X_texts_filtered = [x for x, _ in filtered]
Y_filtered = [y for _, y in filtered]

# ---- SBERT Embedding with Save/Load Logic ----
embedding_file = "sbert_embeddings.npy"
texts_file = "sbert_embedding_texts.pkl"
labels_file = "sbert_embedding_labels.pkl"

# Check if embeddings already exist
if os.path.exists(embedding_file) and os.path.exists(texts_file) and os.path.exists(labels_file):
    print("📂 Loading existing SBERT embeddings...")
    X_emb = np.load(embedding_file)
    
    with open(texts_file, "rb") as f:
        saved_texts = pickle.load(f)
    
    with open(labels_file, "rb") as f:
        saved_labels = pickle.load(f)
    
    # Verify that the saved data matches current data
    if len(saved_texts) == len(X_texts_filtered) and len(saved_labels) == len(Y_filtered):
        print("✅ Loaded existing embeddings successfully!")
    else:
        print("⚠️ Saved data doesn't match current data. Regenerating embeddings...")
        sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
        print("🔄 Generating SBERT embeddings...")
        X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)
        
        # Save the embeddings and corresponding data
        print("💾 Saving SBERT embeddings...")
        np.save(embedding_file, X_emb)
        with open(texts_file, "wb") as f:
            pickle.dump(X_texts_filtered, f)
        with open(labels_file, "wb") as f:
            pickle.dump(Y_filtered, f)
else:
    # Generate embeddings if they don't exist
    sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
    print("🔄 Generating SBERT embeddings...")
    X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)
    
    # Save the embeddings and corresponding data
    print("💾 Saving SBERT embeddings...")
    np.save(embedding_file, X_emb)
    with open(texts_file, "wb") as f:
        pickle.dump(X_texts_filtered, f)
    with open(labels_file, "wb") as f:
        pickle.dump(Y_filtered, f)

# ---- Resampling ----
min_samples = min(Counter(Y_filtered).values())
if min_samples < 2:
    print("⚠️ SMOTE skipped due to insufficient samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered
else:
    smote_k = max(1, min(5, min_samples - 1))
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_emb, Y_filtered)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote)

Y_resampled = to_categorical(Y_resampled_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.25, stratify=np.argmax(Y_resampled, axis=1), random_state=42
)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

class_weights = compute_class_weight("balanced", classes=np.unique(Y_train_int), y=Y_train_int)
class_weight_dict = dict(enumerate(class_weights))

# ---- Model Builder ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout=0.3):
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention = Dropout(dropout)(attention)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention]))
        ff = Dense(ff_dim, activation='relu')(out1)
        ff = Dense(inputs.shape[-1])(ff)
        ff = Dropout(dropout)(ff)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- Train Model ----
gc.collect()
model = build_model(input_shape=X_train.shape[1:], num_classes=Y_resampled.shape[1])
early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, min_lr=1e-6, verbose=1)

In [ ]:
print("🚀 Training model...")
history = model.fit(
    X_train, Y_train_int,
    validation_data=(X_test, Y_test_int),
    epochs=20,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

# ---- Plot Results ----
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.title('Accuracy'); plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Loss'); plt.legend()
plt.tight_layout()
plt.show()

# ---- F1 Score ----
Y_pred_probs = model.predict(X_test, verbose=1)
Y_pred = np.argmax(Y_pred_probs, axis=1)
f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")


model_path = "mood_classifier_model.keras"

# 🧹 Remove old `.keras` model folder if it exists
if os.path.exists(model_path):
    try:
        if os.path.isfile(model_path):
            os.remove(model_path)
        else:
            import shutil
            shutil.rmtree(model_path)
        print(f"🧹 Removed old model: {model_path}")
    except Exception as e:
        print(f"❌ Could not delete old model: {e}")
        raise
import os
import re
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Model (.keras) ----
try:
    model = load_model("mood_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
    print("✅ Model loaded from .keras format!")
except Exception as e:
    print(f"❌ Failed to load .keras model: {e}")
    exit()

# ---- Load SBERT and Resources ----
sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

try:
    with open("mood_keywords.json", "r", encoding="utf-8") as f:
        mood_keywords = json.load(f)
except:
    mood_keywords = {}

# ✅ Load questions from cleaned_questions.json
with open("cleaned_questions.json", "r", encoding="utf-8") as f:
    raw_question_data = json.load(f)

question_data = {}
for entry in raw_question_data:
    category = entry.get("category", "").strip().lower()
    documents = entry.get("questions", [])
    all_questions = []
    for doc in documents:
        q_list = doc.get("questions", [])
        if isinstance(q_list, list):
            clean_qs = [q.strip() for q in q_list if isinstance(q, str) and len(q.strip()) > 10]
            all_questions.extend(clean_qs)
    if all_questions:
        question_data[category] = all_questions

# ---- Logging ----
LOG_FILE = "chat_log.json"

def log_user_query(user_input, mood, question, confidence, answer=None):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_mood": mood,
        "model_confidence": confidence,
        "response_question": question,
        "user_answer": answer
    }
    try:
        if os.path.exists(LOG_FILE):
            with open(LOG_FILE, "r", encoding="utf-8") as f:
                logs = json.load(f)
        else:
            logs = []
        logs.append(entry)
        with open(LOG_FILE, "w", encoding="utf-8") as f:
            json.dump(logs, f, indent=2)
        print("📝 Query logged.")
    except Exception as e:
        print(f"⚠️ Failed to log query: {e}")

# ---- Mood Detection ----
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood.title()
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text) if mood_keywords else None
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood, 1.0

    embedding = sbert.encode([text])
    embedding = np.expand_dims(embedding, axis=1)  # (1, 1, 384)
    prediction = model.predict(embedding, verbose=0)
    mood_index = np.argmax(prediction)
    confidence = float(np.max(prediction))
    mood_label = label_encoder.inverse_transform([mood_index])[0]
    print(f"🤖 Predicted mood: {mood_label} (confidence: {confidence:.2f})")
    return mood_label, confidence

# ---- Fetch Questions by Mood ----
def fetch_questions(mood, count=3):
    mood_key = mood.strip().lower()
    questions = question_data.get(mood_key, [])
    if not questions:
        print(f"⚠️ No questions found for mood '{mood}'.")
        return []
    return list(np.random.choice(questions, size=min(count, len(questions)), replace=False))

# ---- Chatbot Loop ----
def chatbot(num_questions=3):
    print("🧠 MoodBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if not user_input:
            continue
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood, confidence = predict_mood(user_input)
        questions = fetch_questions(mood, count=num_questions)

        if not questions:
            print(f"🧠 MoodBot: I'm here for you. Would you like to talk more about feeling {mood.lower()}?\n")
            continue

        print(f"\n🧠 MoodBot ({mood}): It sounds like you're feeling {mood.lower()}. Let's go through a few questions:\n")
        for i, question in enumerate(questions, 1):
            answer = input(f"{i}. {question}\nYou: ").strip()
            log_user_query(user_input, mood, question, confidence, answer)

        print("\n✅ Thank you for sharing. You can continue chatting or type 'exit' to leave.\n")

# ---- Run Chatbot ----
if __name__ == "__main__":
    chatbot()

# 💾 Save the model using native format
try:
    save_model(model, model_path)
    print(f"✅ Model saved in native Keras format at: {model_path}")
except Exception as e:
    print(f"❌ Error saving model: {e}")

In [ ]:
import os
import re
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Model (.keras) ----
try:
    model = load_model("mood_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
    print("✅ Model loaded from .keras format!")
except Exception as e:
    print(f"❌ Failed to load .keras model: {e}")
    exit()

# ---- Load SBERT and Resources ----
sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

try:
    with open("mood_keywords.json", "r", encoding="utf-8") as f:
        mood_keywords = json.load(f)
except:
    mood_keywords = {}

# ✅ Load and normalize questions from uploaded list-style JSON
with open("questions.json", "r", encoding="utf-8") as f:
    raw_question_data = json.load(f)

# Normalize: Convert list of moods to dict: mood → list of valid questions
question_data = {}
for entry in raw_question_data:
    mood = entry.get("mood", "").strip().lower()
    questions = entry.get("questions", [])
    if mood and isinstance(questions, list):
        valid = [q.strip() for q in questions if isinstance(q, str) and len(q.strip()) > 10]
        if valid:
            question_data.setdefault(mood, []).extend(valid)

# ---- Logging ----
LOG_FILE = "chat_log.json"

def log_user_query(user_input, mood, question, confidence, answer=None):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_mood": mood,
        "model_confidence": confidence,
        "response_question": question,
        "user_answer": answer
    }
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            logs = json.load(f)
    else:
        logs = []
    logs.append(entry)
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)
    print("📝 Query logged.")

# ---- Mood Detection ----
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text) if mood_keywords else None
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood, 1.0

    embedding = sbert.encode([text])
    embedding = np.expand_dims(embedding, axis=1)  # (1, 1, 384)
    prediction = model.predict(embedding, verbose=0)
    mood_index = np.argmax(prediction)
    confidence = float(np.max(prediction))
    mood_label = label_encoder.inverse_transform([mood_index])[0]
    print(f"🤖 Predicted mood: {mood_label} (confidence: {confidence:.2f})")
    return mood_label, confidence

# ---- Fetch Multiple Questions ----
def fetch_questions(mood, count=3):
    mood_key = mood.strip().lower()
    all_questions = question_data.get(mood_key, [])
    if not all_questions:
        print(f"⚠️ No questions found for mood '{mood}'.")
        return []
    return list(np.random.choice(all_questions, size=min(count, len(all_questions)), replace=False))

# ---- Chatbot Main ----
def chatbot(num_questions=3):
    print("🧠 ChatBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood, confidence = predict_mood(user_input)
        questions = fetch_questions(mood, count=num_questions)

        if not questions:
            print(f"🧠 MoodBot: I'm here to support you. Would you like to talk more about feeling {mood.lower()}?\n")
            continue

        print(f"\n🧠 MoodBot ({mood}): I understand you're feeling {mood.lower()}. Let's go through a few questions:\n")

        for i, question in enumerate(questions, 1):
            answer = input(f"{i}. {question}\nYou: ").strip()
            log_user_query(user_input, mood, question, confidence, answer)

        print("\n✅ Thank you for sharing. If you’d like to talk more, just type again or type 'exit' to leave.\n")

# ---- Run ----
if __name__ == "__main__":
    chatbot()


In [ ]:
#Moods_keywords.json
import json
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load the dataset
with open("combined_dataset.json", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Initialize sentiment analyzer
vader = SentimentIntensityAnalyzer()

def infer_mood(text):
    # VADER sentiment
    vader_scores = vader.polarity_scores(text)
    compound = vader_scores["compound"]

    # TextBlob for polarity/subjectivity
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity

    # Rule-based mood inference
    if compound <= -0.5:
        if subjectivity > 0.6:
            return ["Depressed"]
        elif polarity < -0.3:
            return ["Hopeless"]
        else:
            return ["Struggling"]
    elif -0.5 < compound < 0:
        return ["Anxious"]
    elif 0 <= compound < 0.3:
        return ["Neutral"]
    elif 0.3 <= compound <= 0.6:
        return ["Hopeful"]
    else:
        return ["Motivated"]

# Analyze and build output
output_data = []
for entry in data:
    context = entry.get("Context", "")
    response = entry.get("Response", "")
    full_text = f"{context} {response}"

    moods = infer_mood(full_text)

    output_data.append({
        "Context": context.strip(),
        "Detected_Moods": moods,
        "Tags": []
    })

# Save result
with open("moods_keywords.json", "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print("✅ moods_keywords.json generated successfully using auto NLP analysis.")


In [ ]:
#This Code uses Moods_keywords.json file
import os
import json
import gc
import pickle
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model,save_model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, Bidirectional, BatchNormalization, Layer, Add, LayerNormalization, MultiHeadAttention
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score

# ---- Custom Attention Layer ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# ---- Load Dataset ----
with open("combined_dataset.json", "r") as f:
    parsed_data = [json.loads(line) for line in f if line.strip()]
contexts = [sample["Context"] for sample in parsed_data]

# ---- Load Mood Keywords ----
# ---- Load mood samples with Detected_Moods from JSON ----
with open("moods_keywords.json", "r", encoding="utf-8") as f:
    mood_samples = json.load(f)

X_texts, Y_labels = [], []
for item in mood_samples:
    context = item.get("Context", "").strip()
    detected_moods = item.get("Detected_Moods", [])
    if context and detected_moods:
        for mood in detected_moods:
            mood_clean = mood.strip().title()
            X_texts.append(context)
            Y_labels.append(mood_clean)

print(f"📊 Loaded {len(X_texts)} mood-labeled samples from moods_keywords.json")

# ---- Encode and Filter Labels ----
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y_labels)
with open("mood_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

label_counts = Counter(Y_encoded)
valid_labels = {label for label, count in label_counts.items() if count > 1}
filtered = [(x, y) for x, y in zip(X_texts, Y_encoded) if y in valid_labels]
X_texts_filtered = [x for x, _ in filtered]
Y_filtered = [y for _, y in filtered]

# ---- SBERT Embedding with Save/Load Logic ----
embedding_file = "msbert_embeddings.npy"
texts_file = "msbert_embedding_texts.pkl"
labels_file = "msbert_embedding_labels.pkl"

# Check if embeddings already exist
if os.path.exists(embedding_file) and os.path.exists(texts_file) and os.path.exists(labels_file):
    print("📂 Loading existing SBERT embeddings...")
    X_emb = np.load(embedding_file)
    
    with open(texts_file, "rb") as f:
        saved_texts = pickle.load(f)
    
    with open(labels_file, "rb") as f:
        saved_labels = pickle.load(f)
    
    # Verify that the saved data matches current data
    if len(saved_texts) == len(X_texts_filtered) and len(saved_labels) == len(Y_filtered):
        print("✅ Loaded existing embeddings successfully!")
    else:
        print("⚠️ Saved data doesn't match current data. Regenerating embeddings...")
        sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
        print("🔄 Generating SBERT embeddings...")
        X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)
        
        # Save the embeddings and corresponding data
        print("💾 Saving SBERT embeddings...")
        np.save(embedding_file, X_emb)
        with open(texts_file, "wb") as f:
            pickle.dump(X_texts_filtered, f)
        with open(labels_file, "wb") as f:
            pickle.dump(Y_filtered, f)
else:
    # Generate embeddings if they don't exist
    sbert = SentenceTransformer('paraphrase-MiniLM-L12-v2')
    print("🔄 Generating SBERT embeddings...")
    X_emb = sbert.encode(X_texts_filtered, show_progress_bar=True)
    
    # Save the embeddings and corresponding data
    print("💾 Saving SBERT embeddings...")
    np.save(embedding_file, X_emb)
    with open(texts_file, "wb") as f:
        pickle.dump(X_texts_filtered, f)
    with open(labels_file, "wb") as f:
        pickle.dump(Y_filtered, f)

# ---- Resampling ----
min_samples = min(Counter(Y_filtered).values())
if min_samples < 2:
    print("⚠️ SMOTE skipped due to insufficient samples.")
    X_resampled, Y_resampled_encoded = X_emb, Y_filtered
else:
    smote_k = max(1, min(5, min_samples - 1))
    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_emb, Y_filtered)
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled_encoded = smote_tomek.fit_resample(X_smote, Y_smote)

Y_resampled = to_categorical(Y_resampled_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=0.25, stratify=np.argmax(Y_resampled, axis=1), random_state=42
)

X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train_int = np.argmax(Y_train, axis=1)
Y_test_int = np.argmax(Y_test, axis=1)

class_weights = compute_class_weight("balanced", classes=np.unique(Y_train_int), y=Y_train_int)
class_weight_dict = dict(enumerate(class_weights))

# ---- Model Builder ----
def build_model(input_shape, num_classes):
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout=0.3):
        attention = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention = Dropout(dropout)(attention)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention]))
        ff = Dense(ff_dim, activation='relu')(out1)
        ff = Dense(inputs.shape[-1])(ff)
        ff = Dropout(dropout)(ff)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

    inputs = Input(shape=input_shape)
    x = transformer_block(inputs)
    x = transformer_block(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ---- Train Model ----
gc.collect()
model = build_model(input_shape=X_train.shape[1:], num_classes=Y_resampled.shape[1])
early_stop = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.8, patience=7, min_lr=1e-6, verbose=1)

In [ ]:
print("🚀 Training model...")
history = model.fit(
    X_train, Y_train_int,
    validation_data=(X_test, Y_test_int),
    epochs=20,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, Y_test_int, verbose=1)
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")

# ---- Plot Results ----
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs'); plt.ylabel('Accuracy'); plt.title('Accuracy'); plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.title('Loss'); plt.legend()
plt.tight_layout()
plt.show()

# ---- F1 Score ----
Y_pred_probs = model.predict(X_test, verbose=1)
Y_pred = np.argmax(Y_pred_probs, axis=1)
f1 = f1_score(Y_test_int, Y_pred, average="weighted")
print(f"🎯 Weighted F1 Score: {f1 * 100:.2f}%")


model_path = "moods_classifier_model.keras"

# 🧹 Remove old `.keras` model folder if it exists
if os.path.exists(model_path):
    try:
        if os.path.isfile(model_path):
            os.remove(model_path)
        else:
            import shutil
            shutil.rmtree(model_path)
        print(f"🧹 Removed old model: {model_path}")
    except Exception as e:
        print(f"❌ Could not delete old model: {e}")
        raise

# 💾 Save the model using native format
try:
    save_model(model, model_path)
    print(f"✅ Model saved in native Keras format at: {model_path}")
except Exception as e:
    print(f"❌ Error saving model: {e}")

In [ ]:
import os
import re
import json
import pickle
import numpy as np
from datetime import datetime
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# --- Custom Attention Layer ---
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        return K.sum(x * a, axis=1)

# --- Load model & assets ---
try:
    model = load_model("moods_classifier_model.keras", custom_objects={"AttentionLayer": AttentionLayer})
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Model load failed: {e}")
    exit()

sbert = SentenceTransformer("paraphrase-MiniLM-L12-v2")

with open("mood_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

with open("mood_keywords.json", "r", encoding="utf-8") as f:
    mood_keywords = json.load(f)

with open("cleaned_questions.json", "r", encoding="utf-8") as f:
    raw_question_data = json.load(f)

# --- Build question bank from cleaned_questions.json ---
question_data = {}
for entry in raw_question_data:
    category = entry.get("category", "").strip().lower()
    question_data[category] = []
    for doc in entry.get("questions", []):
        for q in doc.get("questions", []):
            if isinstance(q, str) and len(q.strip()) > 10:
                question_data[category].append(q.strip())

# --- Logging ---
def log_response(entry):
    LOG_FILE = "chat_log.json"
    logs = []
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r", encoding="utf-8") as f:
            logs = json.load(f)
    logs.append(entry)
    with open(LOG_FILE, "w", encoding="utf-8") as f:
        json.dump(logs, f, indent=2)

# --- Mood Detection ---
def detect_mood_keywords(text):
    text = text.lower()
    for mood, keywords in mood_keywords.items():
        if any(keyword in text for keyword in keywords):
            return mood
    return None

def predict_mood(text):
    mood = detect_mood_keywords(text)
    if mood:
        print(f"📌 Detected mood from keywords: {mood}")
        return mood, 1.0
    emb = sbert.encode([text])
    emb = np.expand_dims(emb, axis=1)
    pred = model.predict(emb, verbose=0)
    mood_idx = np.argmax(pred)
    confidence = float(np.max(pred))
    mood = label_encoder.inverse_transform([mood_idx])[0]
    print(f"🤖 Predicted mood: {mood} (confidence: {confidence:.2f})")
    return mood, confidence

# --- Fetch Questions ---
def fetch_questions(mood, count=3):
    mood_key = mood.strip().lower()
    available_categories = {k.lower(): k for k in question_data}
    matched_key = available_categories.get(mood_key)

    if not matched_key:
        print(f"⚠️ No questions found for mood '{mood}'.")
        return []

    all_qs = question_data[matched_key]
    return list(np.random.choice(all_qs, size=min(count, len(all_qs)), replace=False))

# --- Main Chatbot ---
def chatbot():
    print("🧠 MoodBot is ready. Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ").strip()
        if not user_input or user_input.lower() in ["exit", "quit"]:
            print("👋 Take care. I'm here whenever you need support.")
            break

        mood, confidence = predict_mood(user_input)
        questions = fetch_questions(mood)

        if not questions:
            print(f"🧠 MoodBot: I'm here to support you. Would you like to share more about how you're feeling?\n")
            continue

        print(f"\n🧠 MoodBot ({mood.title()}): It sounds like you're feeling {mood.lower()}. Let's go through a few questions:\n")
        for i, question in enumerate(questions, 1):
            answer = input(f"{i}. {question}\nYou: ").strip()
            log_response({
                "timestamp": datetime.now().isoformat(),
                "user_input": user_input,
                "predicted_mood": mood,
                "confidence": confidence,
                "question": question,
                "answer": answer
            })

        print("\n✅ Thank you for sharing. You can continue chatting or type 'exit' to leave.\n")

# --- Run ---
if __name__ == "__main__":
    chatbot()
